In [33]:
import numpy as np
import pandas as pd
import model
import nessra_genes_sets_generator as ng
import mygene 
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Matrix and Objective vector preparation

In [2]:
#------------- MATRIX and CLASS VECTOR
matrix = pd.read_csv("~/Desktop/expressionMatrix.tsv", sep = "\t", header=0)
class_vector = pd.read_csv("~/GoogleDrive/DataMiningLab/Model/objective.tsv", sep = "\t", header=0)
print("Matrix shape: {}".format(matrix.shape))
print(matrix.head(2))
print("")

Matrix shape: (60482, 194)
              Gene_ID  TCGA-DM-A28F-01A  TCGA-AD-6889-01A  TCGA-AF-A56L-01A  \
0   ENSG00000270112.3          0.000000          0.004395           0.00000   
1  ENSG00000167578.15          3.703513          2.473260           3.00915   

   TCGA-RU-A8FL-01A  TCGA-AY-4071-01A  TCGA-CM-4751-01A  TCGA-D5-5538-01A  \
0          0.000000          0.000000          0.004063          0.000000   
1          3.857012          5.262676          4.742150          1.759248   

   TCGA-AZ-4616-01A  TCGA-DM-A0X9-01A  ...  TCGA-A6-5657-01A  \
0          0.004094          0.008218  ...          0.000000   
1          3.576053          4.436359  ...          2.401827   

   TCGA-F4-6809-01A  TCGA-A6-6140-01A  TCGA-F4-6805-01A  TCGA-A6-5660-01A  \
0          0.000000          0.000000          0.005296          0.000000   
1          3.362906          2.366821          2.250794          2.382724   

   TCGA-G4-6311-01A  TCGA-CK-6747-01A  TCGA-AF-3913-01A  TCGA-G4-6299-01A  \
0

In [3]:
# transposing matrix
matrix_t = matrix.T
matrix_t.columns = matrix_t.iloc[0]
matrix_t = matrix_t.drop("Gene_ID", axis=0)

# Resetting column names without the version of ENS
col_names = list(matrix_t.columns.values)
new_col_names = []
for name in col_names:
    temp_split = name.split(".")
    new_col_names.append(temp_split[0])
matrix_t.columns = new_col_names
print("Transpose matrix shape: {}".format(matrix_t.shape))
print("")

Transpose matrix shape: (193, 60482)



In [4]:
# class vector 
print("Objective shape: {}".format(class_vector.shape))    

# generating barcode of samples
samples_bar = []
for name in list(matrix_t.index.values):
    samples_bar.append(name[:-4])
fil_for_class = class_vector.barcode.isin(samples_bar)
class_vector = class_vector[fil_for_class]
print("Objective shape after filter for matrix samples: {}".format(class_vector.shape))

# Final vector for classifying
train = class_vector["over2years"]
print("Final vector for classiftibg is a {}, first elements: {}".format(type(train), train[:5]))
print("")

Objective shape: (201, 3)
Objective shape after filter for matrix samples: (193, 3)
Final vector for classiftibg is a <class 'pandas.core.series.Series'>, first elements: 0    1
1    1
2    1
3    1
4    0
Name: over2years, dtype: int64



# NESSRA

importing and preparing output genes

In [7]:
#----------------- NESSRA -----------------
print("----SVM on NESSRA----")
print("")
# importing files output
list_apc = [pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160679_Hs.expansion', sep = ",", header=1),
            pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160679_Hs.interactions', sep = ",", header=1)]
list_smad4 = [pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160680_Hs.expansion', sep = ",", header=1),
            pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160680_Hs.interactions', sep = ",", header=1)]
list_kras = [pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160812_Hs.expansion', sep = ",", header=1),
            [None]]
list_braf = [pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/147717_Hs.expansion', sep = ",", header=1),
            [None]]

# importing tcode conversion matrix
tcode_mat = pd.read_csv('~/GoogleDrive/DataMiningLab/hgnc_filtered_anno.csv', sep = ",", header=0)

----SVM on NESSRA----



In [85]:
# Set filter tresholds and max genes per set here!
tcode_symbol_dict = ng.tcodeSymbolDictGen(tcode_mat)

I'm creating tcode-to-Symbol dictionary, this will take a minute
Done, dictionary ready


In [86]:
# Constructing sets
freq_treshold = 0.3
n_genes_per_file = 500


set_apc = ng.setGen(list_apc[0], list_apc[1],freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=True)
set_smad4 = ng.setGen(list_smad4[0], list_smad4[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)
set_kras = ng.setGen(list_kras[0], list_kras[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)
set_braf = ng.setGen(list_braf[0], list_braf[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)

list_of_sets = [set_apc, set_smad4, set_braf, set_braf ]
print("Len of set apc: {}".format(len(set_apc)))
print("Len of set smad4: {}".format(len(set_smad4)))
print("Len of set kras: {}".format(len(set_kras)))
print("Len of set braf: {}".format(len(set_braf)))

querying 1-500...done.
Finished.
10 input query terms found dup hits:
	[('NKAPP1', 2), ('LINC00237', 2), ('MIR4500HG', 2), ('AQP4-AS1', 2), ('MIR99AHG', 2), ('FTX', 2), ('
13 input query terms found no hit:
	['FAM19A1', 'C8orf46', 'KIAA1468', 'TROVE2', 'FAM57B', 'KIAA1107', 'CTAGE5', 'C14orf159', 'MKL2', 'M
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-500...done.
Finished.
25 input query terms found dup hits:
	[('SDAD1P1', 2), ('MIR4523', 2), ('CHCHD2P6', 2), ('MIR99AHG', 2), ('HCG25', 8), ('URB1-AS1', 2), ('
8 input query terms found no hit:
	['LINC00441', 'PROSC', 'ZCCHC11', 'FAM208A', 'HEXDC', 'SEPT10', 'MARCH5', 'CARS']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-500...done.
Finished.
10 input query terms found dup hits:
	[('LY6G6E', 8), ('MIR21', 2), ('HNF1A-AS1', 2), ('GLIS3-AS1', 2), ('GUSBP6', 2), ('FTH1P23', 2), ('L
7 input query terms found no hit:
	['KIAA0430', 'ISPD', 'ALS2C

In [87]:
ns_mat = model.select_table(matrix_t, list_of_sets)
print("Transpose matrix filtered shape: {}".format(ns_mat.shape))
print(ns_mat.head(2))

Transpose matrix filtered shape: (193, 1352)
                 ENSG00000146083 ENSG00000182141 ENSG00000088179  \
TCGA-DM-A28F-01A         24.7199         1.78346         1.86226   
TCGA-AD-6889-01A         11.7048         1.07085         0.70781   

                 ENSG00000276644 ENSG00000179833 ENSG00000022567  \
TCGA-DM-A28F-01A         8.93392          3.9756         5.12241   
TCGA-AD-6889-01A         4.85309         5.20797         2.86845   

                 ENSG00000114861 ENSG00000138193 ENSG00000112511  \
TCGA-DM-A28F-01A          4.2305         2.21671         7.49493   
TCGA-AD-6889-01A         6.08894         2.43622         13.0324   

                 ENSG00000077044  ... ENSG00000111144 ENSG00000224531  \
TCGA-DM-A28F-01A         6.58233  ...         8.42249         6.12404   
TCGA-AD-6889-01A         4.72495  ...         13.9883         3.13717   

                 ENSG00000152520 ENSG00000160584 ENSG00000164190  \
TCGA-DM-A28F-01A         8.93673         2.58768    

# SVM analysis

splitting dataset, training model and evaluation

In [88]:
#------------ SVM ANALYSIS ------------

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(ns_mat.values, train.values, test_size=0.4,random_state=300) # 80% training and 20% test

In [90]:
X_train.shape





(115, 1352)

In [91]:
# train the model
clf_ns = model.training(X_train, y_train)

y_pred = clf_ns.predict(X_test)


{'C': 10.0, 'gamma': 1.0}
fit_time 0.04608305295308431
score_time 0.06199312210083008
test_accuracy 0.7300944669365723
test_f1 0.8437012263099221
test_precision 0.7300944669365723
test_recall 1.0


In [92]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [93]:
zeros_in_test = len([x for x in y_test if x<1])
1-zeros_in_test/len(y_test)

0.8461538461538461

In [34]:
print("--- Evaluating test set ---")
print("Test matrix shape: {}  \nTest class vector: {} \n".format(X_test.shape, len(y_test)))
print("Frequency filter used in NESSRA output: {}, final number of genes: {}". format(freq_treshold, X_test.shape[1]))
print("")
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

--- Evaluating test set ---
Test matrix shape: (39, 82)  
Test class vector: 39 

Frequency filter used in NESSRA output: 0.9, final number of genes: 82

Accuracy: 0.8205128205128205
Precision: 0.8205128205128205
Recall: 1.0


### Symbol gene printing

In [ ]:
# Save genes as SYMBOL

ens_gene_list = list(ns_mat.columns.values)
mg = mygene.MyGeneInfo()
symbol_genes = mg.querymany(ens_gene_list, scopes='ensembl.gene', fields='symbol', species='human')
gene_list = []
for el in symbol_genes:
    if "symbol" in el:
        gene_list.append(el["symbol"])
print("len of ens genes: {}, len of symbol genes: {}".format(len(ens_gene_list), len(gene_list)))
print(', '.join(gene_list))

In [94]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
model = GaussianNB()

# Train the model using the training sets
model.fit(X_train,y_train)



GaussianNB(priors=None, var_smoothing=1e-09)

In [96]:
#Predict Output
predicted= model.predict(X_test) # 0:Overcast, 2:Mild
print ("Predicted Value:", predicted)

Predicted Value: [1 0 0 1 1 1 0 0 1 1 1 1 0 0 0 1 1 0 0 1 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0 0 0
 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 0 0 1 0
 1 1 0 1]


In [97]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, predicted))

Accuracy: 0.5512820512820513


array([441.806990731, 567.5380872759999, 685.549711422, 534.484250352,
       366.21296002300005, 499.98996420099996, 866.272422593,
       703.102372768, 510.40575742699997, 587.684874819, 5608.79645022,
       493.508861973, 597.695131808, 827.4671856010001, 898.2729642989999,
       528.546901483, 531.7493570319999, 618.130040601, 420.672490477,
       575.04699302, 631.072831163, 579.543994593, 593.761776235,
       469.132029721, 583.407769636, 555.8327420830001, 614.882584246,
       555.659307257, 698.1660873110001, 505.442207005, 554.084706771,
       618.083714312, 815.80986059, 861.228703387, 366.483413064,
       951.0393376510001, 804.1958139689999, 835.10557903, 437.60772505,
       632.1768553640001, 652.67587719, 501.679127561, 364.87406221300006,
       911.9242060060001, 468.21223978300003, 539.117915539,
       564.219586547, 728.901777703, 497.44384709300004,
       331.79619383799997, 472.165057003, 1192.21039593, 583.875470205,
       610.895387473, 332.508753895, 

Help on function fmin in module cma.evolution_strategy:

fmin(objective_function, x0, sigma0, options=None, args=(), gradf=None, restarts=0, restart_from_best='False', incpopsize=2, eval_initial_x=False, parallel_objective=None, noise_handler=None, noise_change_sigma_exponent=1, noise_kappa_exponent=0, bipop=False, callback=None)
    functional interface to the stochastic optimizer CMA-ES
    for non-convex function minimization.
    
    Calling Sequences
    ``fmin(objective_function, x0, sigma0)``
        minimizes ``objective_function`` starting at ``x0`` and with
        standard deviation ``sigma0`` (step-size)
    ``fmin(objective_function, x0, sigma0, options={'ftarget': 1e-5})``
        minimizes ``objective_function`` up to target function value 1e-5,
        which is typically useful for benchmarking.
    ``fmin(objective_function, x0, sigma0, args=('f',))``
        minimizes ``objective_function`` called with an additional
        argument ``'f'``.
    ``fmin(objective_func